In [2]:
import pandas as pd
import pyodbc
import requests
from key import API_key

In [17]:
def validate_numbers(df):
    base_url = "https://api.realvalidation.com/rpvWebService/RealPhoneValidationTurbo.php"
    output = "json"
    results_dict_list = []
    status = 0
    for row in df.itertuples():
        new_dict = {}
        phone = row.OFFICE_TELEPHONE
        new_dict["OFFICE_TELEPHONE"] = phone
        parameters = {'output':output, 'phone' : phone, 'token': API_key}
        response =  requests.get(base_url, params=parameters)
        results = response.json()
        status = results['status']
        new_dict['status']=status
        results_dict_list.append(new_dict)
    new_df = pd.DataFrame(results_dict_list)
    final_df = pd.merge(new_df, df, on='OFFICE_TELEPHONE')
    return(final_df)

In [4]:
phone_df = pd.read_csv('phone_test.csv')
phone_df.head()

,OFFICE_TELEPHONE,COMMENTS,WSLIVE_FILE_DT
0,2000000000,NOT IN SERVICE,7/5/2019
1,2037318725,NOT IN SERVICE,7/5/2019
2,2098230083,NOT IN SERVICE,7/5/2019
3,2102571400,NOT IN SERVICE,7/5/2019
4,2104959560,NOT IN SERVICE,7/5/2019


In [3]:
validated_df = validate_numbers(phone_df)
validated_df.head()

,OFFICE_TELEPHONE,COMMENTS,WSLIVE_FILE_DT,status
0,2000000000,NOT IN SERVICE,7/5/2019,invalid-phone
1,2037318725,NOT IN SERVICE,7/5/2019,disconnected
2,2098230083,NOT IN SERVICE,7/5/2019,disconnected-70
3,2102571400,NOT IN SERVICE,7/5/2019,disconnected
4,2104959560,NOT IN SERVICE,7/5/2019,disconnected


In [20]:
validated_df.to_csv("real_phone_validated_results.csv")

In [4]:
validated_df.groupby(['WSLIVE_FILE_DT','status']).count()

OFFICE_TELEPHONE  COMMENTS
WSLIVE_FILE_DT status                                     
7/19/2019      connected                      13        13
               disconnected                    7         7
               disconnected-70                 5         5
7/5/2019       connected                      10        10
               disconnected                   10        10
               disconnected-70                 4         4
               invalid-phone                   1         1